In [1]:
import json
import pandas as pd
import requests
import os

from pathlib import Path
from typing import Union
from collections import defaultdict
from sqlalchemy import create_engine


In [2]:
# see website for information:
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/

api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
               'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
               'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Variable to store the total number of electric vehicle stations in the US
total_stations_us = 0

# Dictionary to store the totals for each state
state_totals = {}

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the total number of electric vehicle stations for the specified state
        total_stations = data.get('total_results', 0)

        # Increment the total stations in the US variable
        total_stations_us += total_stations

        # Store the result in the dictionary
        state_totals[state_code] = total_stations

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Sort the dictionary by state in decreasing order based on total stations
sorted_totals = dict(sorted(state_totals.items(), key=lambda item: item[1], reverse=True))

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the sorted totals to a JSON file in the 'data_query' folder
state_data_file_path = os.path.join(output_folder, 'electric_stations_by_state.json')
with open(state_data_file_path, 'w', encoding='utf-8') as state_data_file:
    json.dump(sorted_totals, state_data_file, ensure_ascii=False, indent=4)

# Print the total number of electric vehicle stations in the US
print(f'Total electric vehicle stations in the US for fuel type ELEC: {total_stations_us}')

print(f'The sorted totals have been saved to: {state_data_file_path}')


Total electric vehicle stations in the US for fuel type ELEC: 79449
The sorted totals have been saved to: data_query/electric_stations_by_state.json


In [3]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of all US state codes
state_codes = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
    'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# List of attributes to retrieve, including the new ones
attributes_to_retrieve = [
    'id', 'station_name', 'street_address', 'city', 'state', 'zip',
    'latitude', 'longitude', 'owner_type_code', 'ev_connector_types', 'ev_pricing',
    'geocode_status', 'access_code', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse'
]

# List to store detailed information for stations meeting the criteria
filtered_stations = []

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'restricted_access': 'false',  # Add restricted_access parameter
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Add detailed information for each station meeting the criteria
        filtered_stations.extend([
            {attr: station[attr] for attr in attributes_to_retrieve}
            for station in data.get('fuel_stations', [])
        ])

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Print the total count of electric vehicle stations in the US after filtering
total_us_stations = sum(len(data.get('fuel_stations', [])) for state_code in state_codes)
print(f'Total electric vehicle stations in the US after filtering: {total_us_stations}')

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the filtered stations to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'filtered_stations_attributes.json')
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(filtered_stations, output_file, ensure_ascii=False, indent=4)
    
    
print(f'The filtered stations with specified attributes have been saved to: {output_file_path}')
        

Total electric vehicle stations in the US after filtering: 7100
The filtered stations with specified attributes have been saved to: data_query/filtered_stations_attributes.json


In [4]:
# Read in JSON previously created from API request
output_file_path = Path("data_query/filtered_stations_attributes.json")

# Load data from the JSON file
if output_file_path.exists():
    with output_file_path.open("r", encoding="utf8") as f:
        station_data = json.load(f)

# Create a DataFrame from the loaded data
station_df = pd.DataFrame(station_data)

# Organize the columns in the specified sequence
station_df = station_df[['id', 'station_name', 'street_address', 'city', 'state', 'zip',
         'latitude', 'longitude', 'owner_type_code', 'ev_connector_types', 'ev_pricing',
         'geocode_status', 'access_code', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse']]

# Display the DataFrame
# print(station_df.columns)

station_df.head()


,id,station_name,street_address,city,state,zip,latitude,longitude,owner_type_code,ev_connector_types,ev_pricing,geocode_status,access_code,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,ev_other_evse
0,17,Spire - Montgomery Operations Center,2951 Chestnut St,Montgomery,AL,36107,32.367916,-86.267021,T,None,None,200-9,private,NaN,NaN,NaN,None
1,597,Spire,2828 Dauphin St,Mobile,AL,36606,30.689832,-88.108861,T,None,None,200-9,private,NaN,NaN,NaN,None
2,13712,Gala Gas Co Inc,54 Old US Highway 82,Eufaula,AL,36027,31.912022,-85.150318,P,None,None,GPS,private,NaN,NaN,NaN,None
3,13723,Superior Gas,421 Noble St,Anniston,AL,36201,33.642079,-85.828466,P,None,None,GPS,public,NaN,NaN,NaN,None
4,13725,Superior Gas,702 Memorial Dr,Piedmont,AL,36272,33.921589,-85.620314,P,None,None,200-9,public,NaN,NaN,NaN,None


In [8]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# Set the state parameter to 'CA' for California
state_code = 'CA'

# List of attributes to retrieve for map plotting
attributes_to_retrieve = [
    'id', 'station_name', 'street_address', 'city', 'state', 'zip',
    'latitude', 'longitude', 'owner_type_code', 'ev_connector_types', 'ev_pricing',
    'geocode_status', 'access_code'
]

# List to store detailed information for stations meeting the criteria
filtered_stations = []

query_params = {
    'fuel_type_code': 'ELEC',
    'state': state_code,
    'country': 'US',
    'restricted_access': 'false',  # Add restricted_access parameter
    'api_key': api_key
}

# Make the API request
response = requests.get(base_url, params=query_params)

# Parse the JSON response
data = response.json()

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Add detailed information for each station meeting the criteria
    for station in data.get('fuel_stations', []):
        # Check if all essential attributes have usable values
        if all(
            station.get(attr) is not None and station.get(attr) != ''
            for attr in attributes_to_retrieve
        ):
            # Combine 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse'
            evse_combined = {
                'evse_combined': {
                    'ev_level1': station.get('ev_level1_evse_num', 0),
                    'ev_level2': station.get('ev_level2_evse_num', 0),
                    'ev_dc_fast': station.get('ev_dc_fast_num', 0),
                    'ev_other': station.get('ev_other_evse', 0)
                }
            }
            station.update(evse_combined)

            # Remove the individual evse attributes from the station
            for evse_attr in ['ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse']:
                del station[evse_attr]

            filtered_stations.append({attr: station[attr] for attr in attributes_to_retrieve})

else:
    # Print an error message if the request was not successful
    print(f'Error: {response.status_code} - {response.text}')

# Create a DataFrame from the loaded data
df = pd.DataFrame(filtered_stations)

# Display the DataFrame
print(df)

# Print the total count of electric vehicle stations in California after filtering
total_ca_stations = len(df)
print(f'Total electric vehicle stations in California after filtering: {total_ca_stations}')

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the filtered stations to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'filtered_stations_combined_evse_CA.json')
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(filtered_stations, output_file, ensure_ascii=False, indent=4)

print(f'The filtered stations with combined EVSE attributes in California have been saved to: {output_file_path}')


          id                                   station_name  \
0       1523                  Los Angeles Convention Center   
1       1556                            LADWP - Sylmar West   
2       1573                         LADWP - Fairfax Center   
3       1583                 California Air Resources Board   
4       6355                         Scripps Green Hospital   
...      ...                                            ...   
1763  323695  El Dorado County Department of Transportation   
1764  323696                             Georgetown Library   
1765  323697                 El Dorado County Senior Center   
1766  323703                                 7Charge - Napa   
1767  323706                         7Charge - Walnut Creek   

               street_address          city state    zip   latitude  \
0          1201 S Figueroa St   Los Angeles    CA  90015  34.040539   
1        13201 Sepulveda Blvd        Sylmar    CA  91342  34.303090   
2          2311 S Fairfax Ave 